In [61]:
import json
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as la
import pandas as pd
import re
import networkx as nx

In [6]:
colors = pd.read_csv(f'/work/ccnr/glover.co/net_design/NetDesign/data/lego/colors.csv',delimiter=',')
elements = pd.read_csv(f'/work/ccnr/glover.co/net_design/NetDesign/data/lego/elements.csv',delimiter=',')
inventories = pd.read_csv(f'/work/ccnr/glover.co/net_design/NetDesign/data/lego/inventories.csv',delimiter=',')
inventory_minifigs = pd.read_csv(f'/work/ccnr/glover.co/net_design/NetDesign/data/lego/inventory_minifigs.csv',delimiter=',')
inventory_parts = pd.read_csv(f'/work/ccnr/glover.co/net_design/NetDesign/data/lego/inventory_parts.csv',delimiter=',')
inventory_sets = pd.read_csv(f'/work/ccnr/glover.co/net_design/NetDesign/data/lego/inventory_sets.csv',delimiter=',')
minifigs = pd.read_csv(f'/work/ccnr/glover.co/net_design/NetDesign/data/lego/minifigs.csv',delimiter=',')
part_categories = pd.read_csv(f'/work/ccnr/glover.co/net_design/NetDesign/data/lego/part_categories.csv',delimiter=',')
part_relationships = pd.read_csv(f'/work/ccnr/glover.co/net_design/NetDesign/data/lego/part_relationships.csv',delimiter=',')
parts = pd.read_csv(f'/work/ccnr/glover.co/net_design/NetDesign/data/lego/parts.csv',delimiter=',')
sets = pd.read_csv(f'/work/ccnr/glover.co/net_design/NetDesign/data/lego/sets.csv',delimiter=',')
themes = pd.read_csv(f'/work/ccnr/glover.co/net_design/NetDesign/data/lego/themes.csv',delimiter=',')

In [7]:
def extract_sizes(name):
    match = re.search(r'(\d+)\s*[xX]\s*(\d+)(?:\s*[xX]\s*(\d+))?', name)
    if match:
        x, y, z = match.groups()
        return int(x), int(y), int(z) if z else 1  # Default z to 1
    return None, None, None 

In [8]:
parts[['x','y','z']] = parts['name'].apply(lambda name: pd.Series(extract_sizes(name)))

In [ ]:
with open('../../data/lego/eval_datasets/classics/000012/info.json','r') as f:
    data =json.load(f)

In [100]:
df = pd.read_pickle('../../data/lego/eval_datasets/classics/000012/conns.pkl')

In [101]:
df

[(),
 (array([[-0.5,  3. ,  2.5],
         [ 0.5,  3. ,  2.5],
         [ 0.5,  3. , -1.5],
         [-0.5,  3. ,  1.5],
         [-0.5,  3. , -2.5],
         [ 0.5,  3. , -2.5],
         [ 0.5,  3. ,  0.5],
         [ 0.5,  3. ,  1.5],
         [-0.5,  3. ,  0.5],
         [-0.5,  3. , -0.5],
         [ 0.5,  3. , -0.5],
         [-0.5,  3. , -1.5]], dtype=float32),
  array([[3.0751416e+02, 3.8053992e+02, 4.2823367e-02],
         [2.9017609e+02, 3.7187091e+02, 4.3417163e-02],
         [2.2082390e+02, 4.0654700e+02, 4.1041989e-02],
         [2.9017609e+02, 3.8920892e+02, 4.2229574e-02],
         [2.2082390e+02, 4.2388501e+02, 3.9854400e-02],
         [2.0348586e+02, 4.1521600e+02, 4.0448196e-02],
         [2.5550000e+02, 3.8920892e+02, 4.2229574e-02],
         [2.7283804e+02, 3.8053992e+02, 4.2823367e-02],
         [2.7283804e+02, 3.9787796e+02, 4.1635782e-02],
         [2.5550000e+02, 4.0654700e+02, 4.1041989e-02],
         [2.3816196e+02, 3.9787796e+02, 4.1635782e-02],
         [2.38

In [55]:
brick_part_names = []

In [56]:
steps = data['operations'].keys()
for s in steps:
    bricks = data['operations'][s]['bricks']
    for b in bricks:
        try:
            brick_part_names.append(b['brick_type'])
        except:
            continue

In [57]:
parts[parts['part_num'].isin(brick_part_names)]

,part_num,name,part_cat_id,part_material,x,y,z
8395,2431,Tile 1 x 4 with Groove,19,Plastic,1.0,4.0,1.0
11799,3001,Brick 2 x 4,11,Plastic,2.0,4.0,1.0
11892,3002,Brick 2 x 3,11,Plastic,2.0,3.0,1.0
13464,3020,Plate 2 x 4,14,Plastic,2.0,4.0,1.0
13471,3021,Plate 2 x 3,14,Plastic,2.0,3.0,1.0
13481,3022,Plate 2 x 2,14,Plastic,2.0,2.0,1.0
13514,3024,Plate 1 x 1,14,Plastic,1.0,1.0,1.0
13651,3030,Plate 4 x 10,14,Plastic,4.0,10.0,1.0
13661,3032,Plate 4 x 6,14,Plastic,4.0,6.0,1.0
15726,3069b,Tile 1 x 2 with Groove,19,Plastic,1.0,2.0,1.0


In [69]:
g = nx.Graph()

In [70]:
def is_connected(brick1,brick2):
    pass

In [83]:
steps = data['operations'].keys()
for s in steps:
    bricks = data['operations'][s]['bricks']
    for b in bricks:
        # print(b)
        # node_attributes = {'type':None,'position':None,'keypoint':None}
        try:
        # node_attributes['type'] = b['brick_type']
        # node_attributes['position'] = b['brick_transform']['position']
        # node_attributes['keypoint'] = b['brick_transform']['keypoint_brick']
            g.add_node(g.number_of_nodes(),brick_type =  b['brick_type'], position = b['brick_transform']['position'], keypoint = b['brick_transform']['keypoint_brick'])
        except:
            continue

In [104]:
colors.head(20)

,id,name,rgb,is_trans
0,-1,[Unknown],0033B2,False
1,0,Black,05131D,False
2,1,Blue,0055BF,False
3,2,Green,237841,False
4,3,Dark Turquoise,008F9B,False
5,4,Red,C91A09,False
6,5,Dark Pink,C870A0,False
7,6,Brown,583927,False
8,7,Light Gray,9BA19D,False
9,8,Dark Gray,6D6E5C,False
